In [1]:
import GetOldTweets3 as got
#using getoldtweets library to get twitter data.

In [ ]:
#getting tweets by username
tweetCriteria = got.manager.TweetCriteria().setUsername('orangebook_').setMaxTweets(100)
                                                                                    
tweet = got.manager.TweetManager.getTweets(tweetCriteria)

t = [[twee.text] for twee in tweet]
for i in range (0,100):
    print(t[i])
    



In [ ]:
import pandas as pd
t = pd.DataFrame(t)

t.to_csv(r'C:\Users\acer pc\OneDrive\Desktop\orangebook\orangebook.csv')

In [2]:
tweetCriteria = got.manager.TweetCriteria().setUsername('navalbot').setMaxTweets(100)
                                                                                    
tweet = got.manager.TweetManager.getTweets(tweetCriteria)

t = [[twee.text] for twee in tweet]
for i in range (0,100):
    print(t[i])

import pandas as pd

t = pd.DataFrame(t)
t.to_csv(r'C:\Users\acer pc\OneDrive\Desktop\orangebook\orangebook.csv')
    

['"Changing your interpretation of your past is often just as good as changing your past." - @naval']
['Thanks for giving credit ']
['"A contrarian isn’t one who always objects - that’s a conformist of a different sort. A contrarian reasons independently, from the ground up, and resists pressure to conform." - @naval']
['"Work as hard as you can. Even though who you work with and what you work on are more important than how hard you work." - @naval']
['"Involve groups in the creative process and you inevitably end up with a pile of pleasing, socially-acceptable lies. Contrast blockbuster movies with the great books." - @naval']
['"If you’re more passionate about founding a business than the business itself, you can fall into a ten year trap. Better to stay emotionally unattached and select the best opportunity that arises." - @naval']
['"You will get rich by giving society what it wants but does not yet know how to get. At scale." - @naval']
['"If friends spoke to us like politicians a

In [3]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [6]:
data_new = (str(t))

In [7]:
print(data_new)

                                                    0
0   "Changing your interpretation of your past is ...
1                           Thanks for giving credit 
2   "A contrarian isn’t one who always objects - t...
3   "Work as hard as you can. Even though who you ...
4   "Involve groups in the creative process and yo...
..                                                ...
95  "Don't write to make money, write to build rel...
96  "Before you can lie to another, you must first...
97  "Politics is the exercise of power without mer...
98  "The secret to public speaking is to speak as ...
99  "Twitter is a way to broadcast messages to hun...

[100 rows x 1 columns]


In [ ]:
import re 

def text_cleaner(text):
    newString = text.lower()
    newString = re.sub(r"'s\b","",newString)
    
    newString = re.sub("[^a-zA-Z]", " ", newString) 

    long_words = []
    
    for i in newString.split():
        if len(i) >= 3:
            long_words.append(i)
    return (' '.join(long_words)).strip()

data_new = text_cleaner(data_text)

In [8]:
print(data_new)

                                                    0
0   "Changing your interpretation of your past is ...
1                           Thanks for giving credit 
2   "A contrarian isn’t one who always objects - t...
3   "Work as hard as you can. Even though who you ...
4   "Involve groups in the creative process and yo...
..                                                ...
95  "Don't write to make money, write to build rel...
96  "Before you can lie to another, you must first...
97  "Politics is the exercise of power without mer...
98  "The secret to public speaking is to speak as ...
99  "Twitter is a way to broadcast messages to hun...

[100 rows x 1 columns]


In [9]:
def create_seq(text):
    length = 30
    sequences = list()
    for i in range(length, len(text)):
        seq = text[i-length:i+1]
        sequences.append(seq)
    print('total sequences: %d' %len(sequences))
    return sequences

sequences = create_seq(data_new)

total sequences: 641


In [10]:
chars = sorted(list(set(data_new)))
mapping = dict((c, i) for i, c in enumerate(chars))

def encode_seq(seq):
    sequences = list()
    for line in seq:
        # integer encode line
        encoded_seq = [mapping[char] for char in line]
        # store
        sequences.append(encoded_seq)
    return sequences

# encode the sequences
sequences = encode_seq(sequences)

In [11]:
from sklearn.model_selection import train_test_split

# vocabulary size
vocab = len(mapping)
sequences = np.array(sequences)
# create X and y
X, y = sequences[:,:-1], sequences[:,-1]
# one hot encode y
y = to_categorical(y, num_classes=vocab)
# create train and validation sets
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

print('Train shape:', X_tr.shape, 'Val shape:', X_val.shape)

Train shape: (576, 30) Val shape: (65, 30)


In [12]:
model = Sequential()
model.add(Embedding(vocab, 50, input_length=30, trainable=True))
model.add(GRU(150, recurrent_dropout=0.1, dropout=0.1))
model.add(Dense(vocab, activation='softmax'))
print(model.summary())

# compile the model
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')
# fit the model
model.fit(X_tr, y_tr, epochs=100, verbose=2, validation_data=(X_val, y_val))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 50)            2650      
_________________________________________________________________
gru_1 (GRU)                  (None, 150)               90450     
_________________________________________________________________
dense_1 (Dense)              (None, 53)                8003      
Total params: 101,103
Trainable params: 101,103
Non-trainable params: 0
_________________________________________________________________
None

Train on 576 samples, validate on 65 samples
Epoch 1/100
 - 1s - loss: 3.7199 - acc: 0.2847 - val_loss: 3.2340 - val_acc: 0.2769
Epoch 2/100
 - 0s - loss: 3.0337 - acc: 0.3142 - val_loss: 3.0607 - val_acc: 0.2769
Epoch 3/100
 - 0s - loss: 2.8873 - acc: 0.3142 - val_loss: 2.9404 - val_acc: 0

Epoch 82/100
 - 1s - loss: 0.0908 - acc: 0.9913 - val_loss: 3.3046 - val_acc: 0.4308
Epoch 83/100
 - 0s - loss: 0.0973 - acc: 0.9861 - val_loss: 3.3056 - val_acc: 0.3385
Epoch 84/100
 - 0s - loss: 0.1041 - acc: 0.9896 - val_loss: 3.3019 - val_acc: 0.4308
Epoch 85/100
 - 0s - loss: 0.1060 - acc: 0.9878 - val_loss: 3.2723 - val_acc: 0.4308
Epoch 86/100
 - 1s - loss: 0.0825 - acc: 0.9965 - val_loss: 3.2769 - val_acc: 0.4462
Epoch 87/100
 - 1s - loss: 0.0847 - acc: 0.9878 - val_loss: 3.2647 - val_acc: 0.4462
Epoch 88/100
 - 0s - loss: 0.0780 - acc: 0.9965 - val_loss: 3.2993 - val_acc: 0.4308
Epoch 89/100
 - 0s - loss: 0.0729 - acc: 0.9948 - val_loss: 3.3301 - val_acc: 0.3538
Epoch 90/100
 - 1s - loss: 0.0651 - acc: 0.9965 - val_loss: 3.3130 - val_acc: 0.4462
Epoch 91/100
 - 1s - loss: 0.0951 - acc: 0.9896 - val_loss: 3.3786 - val_acc: 0.3231
Epoch 92/100
 - 1s - loss: 0.0764 - acc: 0.9913 - val_loss: 3.4051 - val_acc: 0.3385
Epoch 93/100
 - 1s - loss: 0.0724 - acc: 0.9931 - val_loss: 3.442

In [13]:
# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
	in_text = seed_text
	# generate a fixed number of characters
	for _ in range(n_chars):
		# encode the characters as integers
		encoded = [mapping[char] for char in in_text]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict character
		yhat = model.predict_classes(encoded, verbose=0)
		# reverse map integer to character
		out_char = ''
		for char, index in mapping.items():
			if index == yhat:
				out_char = char
				break
		# append to input
		in_text += char
	return in_text

In [23]:
inp = 'broadcast messages'
print(len(inp))


18


In [24]:
generate_seq(model,mapping,30,inp.lower(),15)

'broadcast messages to hun...\n\n[10'